# 벡터 유사도

## 1. 코사인 유사도

Similarity = cos(세타) = A•B / ||A|| ||B||

In [2]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
#코사인 유사도 계산
def cos_sim(A: np.array ,B: np.array):
    return dot(A,B)/(norm(A)*norm(B))

문서1 : 저는 사과 좋아요  
문서2 : 저는 바나나 좋아요  
문서3 : 저는 바나나 좋아요 저는 바나나 좋아요  

In [21]:
import pandas as pd

docs = [
  '저는 사과 좋아요',
  '저는 바나나 좋아요',
  '저는 바나나 좋아요 저는 바나나 좋아요'
] 
#전체 단어 집합 생성
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()
vocab

['바나나', '사과', '저는', '좋아요']

In [60]:
#DTM 구하기
DTM=[]
for doc in docs:
    vector=[]
    for voca in vocab:
        #각 단어에 대해 문장에서 몇번 등장하는지 계산(TF)
        vector.append(doc.count(voca))
    DTM.append(vector)
df = pd.DataFrame(DTM,['문서1','문서2','문서3'],columns= vocab)
df

,바나나,사과,저는,좋아요
문서1,0,1,1,1
문서2,1,0,1,1
문서3,2,0,2,2


In [71]:
doc1=DTM[0]
doc2=DTM[1]
doc3=DTM[2]

In [64]:
print(cos_sim(doc1, doc2)) #문서1과 문서2의 코사인 유사도
print(cos_sim(doc1, doc3)) #문서1과 문서3의 코사인 유사도
print(cos_sim(doc2, doc3)) #문서2과 문서3의 코사인 유사도

0.6666666666666667
0.6666666666666667
1.0000000000000002


cos_sim(1,2)와 cos_sim(1,3)의 유사도가 같음  
-> 문서의 길이가 다른 상황에서도 공정한 비교 가능  
벡터의 크기가 아닌, 방향(패턴)에 초점을 두기 때문

## 코사인 유사도를 이용한 추천 시스템 구현하기

In [31]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [35]:
data = pd.read_csv('data/movies_metadata.csv', low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [37]:
#상위 20000개의 데이터 사용
data = data.head(20000)

In [38]:
#NULL값 확인
data['overview'].isnull().sum()

135

In [41]:
# overview에서 Null값 제거(빈 값(empty value)으로 대체)
data['overview'] = data['overview'].fillna('')

In [43]:
tfidf = TfidfVectorizer(stop_words='english')
# overview에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['overview'])
print(tfidf_matrix.shape)

(20000, 47487)


In [44]:
#코사인 유사도 구하기
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [48]:
#타이틀과 인덱스를 갖는 테이블 생성
indices = pd.Series(data.index, index=data['title']).drop_duplicates()
print(indices.head())

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [47]:
idx = indices['Father of the Bride Part II']
print(idx)

4


In [57]:
# 가장 overview가 비슷한 10개의 영화를 찾아내는 함수
def get_recommendations(title: str, cosine_sim = cosine_sim):
    #선택한 영화의 인덱스를 받아옵니다.
    idx = indices[title]
    
    #모든 영화에 대해 해당 영화와의 유사도 구하기
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    #유사도에 따라 영화들을 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    #가장 유사한 10개의 영화 선택하고 인덱스 받아오기
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    
    #제목 리턴
    return data['title'].iloc[movie_indices]

In [58]:
get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object

## 2. 유클리드 거리(Euclidean distance)

두 점 사이의 거리

In [59]:
import numpy as np
def dist(x,y):   
    return np.sqrt(np.sum((x-y)**2))

In [72]:
doc1=DTM[0]
doc2=DTM[1]
doc3=DTM[2]
docQ = np.array((1,1,0,1))

In [73]:
print(dist(doc1,docQ))
print(dist(doc2,docQ))
print(dist(doc3,docQ))

1.4142135623730951
1.4142135623730951
2.6457513110645907


값이 작을수록 거리가 가깝다 -> 유사하다

## 3. 자카드 유사도(Jaccard similarity)

J(A,B) = |AnB| / |AuB| (합집합에서 교집합의 비율, 동일하면 1, 전부 다르면 0)

In [76]:
# 다음과 같은 두 개의 문서가 있습니다.
# 두 문서 모두에서 등장한 단어는 apple과 banana 2개.
doc1 = "apple banana everyone like likey watch card holder"
doc2 = "apple banana coupon passport love you"

# 토큰화
tokenized_doc1 = doc1.split()
tokenized_doc2 = doc2.split()
print(tokenized_doc1)
print(tokenized_doc2)

['apple', 'banana', 'everyone', 'like', 'likey', 'watch', 'card', 'holder']
['apple', 'banana', 'coupon', 'passport', 'love', 'you']


In [79]:
#합집합
union = set(tokenized_doc1).union(set(tokenized_doc2))
print(len(union),union)

12 {'like', 'passport', 'everyone', 'love', 'card', 'you', 'banana', 'coupon', 'watch', 'likey', 'holder', 'apple'}


In [83]:
#교집합
intersection = set(tokenized_doc1).intersection(set(tokenized_doc2))
print(len(intersection),intersection)

2 {'banana', 'apple'}


In [84]:
print(len(intersection)/len(union))

0.16666666666666666


## 4. 레벤슈타인 거리(Levenshtein Distance)

레벨슈타인 거리: 문자열이 얼마나 비슷한지를 나타내는 것으로 편집거리라고도 부른다.  
ex) hat -> here (편집거리3)  
a -> e  
t -> r  
  -> e (+e)

In [1]:
#레벨슈타인 거리 구하기
def calc_distance(a: str , b: str):
    if a==b: return 0
    a_len=len(a)
    b_len=len(b)
    if a=="" : return b_len
    if b=="" : return a_len

    #2차원 매트릭스 (a_len+1, b_len+1)
    matrix=[[0 for j in range(b_len+1)] for i in range(a_len+1)]

    #첫 번째 행, 첫 번째 열을  문자열 길이로 초기화
    for i in range(a_len+1):
        matrix[i][0]=i
    for j in range(b_len+1):
        matrix[0][j]=j

    for i in range(1, a_len+1):
        ac=a[i-1] #비교할 문자
        for j in range(1, b_len+1):
            bc=b[j-1] #비교할 문자
            cost = 0 if (ac==bc) else 1 #같으면 비용 0, 다르면 비용 1
            matrix[i][j]=min([
                matrix[i-1][j]+1, #문자 삽입
                matrix[i][j-1]+1, #문자 제거
                matrix[i-1][j-1]+cost #문자 변경
            ])

    return matrix[a_len][b_len]

In [2]:
#example
samples = ["신촌역","신천군","신천역","신발","마곡역"]
base = samples[0]
r = sorted(samples, key = lambda x : calc_distance(base,x))
for x in r:
    print(calc_distance(base,x),x)

0 신촌역
1 신천역
2 신천군
2 신발
2 마곡역


In [8]:
def levenshtein(s1, s2, debug=False):
    if len(s1) < len(s2):
        return levenshtein(s2, s1, debug)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 #문자 삽입
            deletions = current_row[j] + 1 #문자 제거
            substitutions = previous_row[j] + (c1 != c2) #문자 변경
            current_row.append(min(insertions, deletions, substitutions))

        if debug:
            print(current_row[1:])

        previous_row = current_row

    return previous_row[-1]

In [9]:
s1 = '꿈을꾸는아이'
s2 = '아이오아이'
levenshtein(s1, s2, debug=True)

[1, 2, 3, 4, 5]
[2, 2, 3, 4, 5]
[3, 3, 3, 4, 5]
[4, 4, 4, 4, 5]
[4, 5, 5, 4, 5]
[5, 4, 5, 5, 4]


4

### 한글처럼 한 글자가 요소들로 이루어진 언어에 적합한 방식인가?

In [16]:
s1 = '서비스'
s2 = '써비스'
s3 = '자비스'

s4 = '밥 먹어'
s5 = '밥 먹엉'
s6 = '너 먹어'

print(levenshtein(s1,s2),levenshtein(s1,s3))
print(levenshtein(s4,s5),levenshtein(s4,s6))

1 1
1 1


## 한글 텍스트 초성/중성/종성 분리하기

In [3]:
kor_begin = 44032
kor_end = 55203
chosung_base = 588
jungsung_base = 28
jaum_begin = 12593
jaum_end = 12622
moum_begin = 12623
moum_end = 12643

chosung_list = [ 'ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 
        'ㅅ', 'ㅆ', 'ㅇ' , 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

jungsung_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 
        'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 
        'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 
        'ㅡ', 'ㅢ', 'ㅣ']

jongsung_list = [
    ' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ',
        'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 
        'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 
        'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

jaum_list = ['ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄸ', 'ㄹ', 
              'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 
              'ㅃ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

moum_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 
              'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']

def compose(chosung, jungsung, jongsung):
    char = chr(
        kor_begin +
        chosung_base * chosung_list.index(chosung) +
        jungsung_base * jungsung_list.index(jungsung) +
        jongsung_list.index(jongsung)
    )
    return char

def decompose(c):
    if not character_is_korean(c):
        return None
    i = ord(c)
    if (jaum_begin <= i <= jaum_end):
        return (c, ' ', ' ')
    if (moum_begin <= i <= moum_end):
        return (' ', c, ' ')

    # decomposition rule
    i -= kor_begin
    cho  = i // chosung_base
    jung = ( i - cho * chosung_base ) // jungsung_base 
    jong = ( i - cho * chosung_base - jung * jungsung_base )    
    return (chosung_list[cho], jungsung_list[jung], jongsung_list[jong])

def character_is_korean(c):
    i = ord(c)
    return ((kor_begin <= i <= kor_end) or
            (jaum_begin <= i <= jaum_end) or
            (moum_begin <= i <= moum_end))

In [5]:
decompose('감')

('ㄱ', 'ㅏ', 'ㅁ')

In [6]:
compose('ㄲ', 'ㅜ', 'ㅁ')

'꿈'

## 초성/중성/종성 분리를 적용한 Levenshtein distance

In [7]:
def jamo_levenshtein(s1, s2, debug=False):
    if len(s1) < len(s2):
        return jamo_levenshtein(s2, s1, debug)

    if len(s2) == 0:
        return len(s1)

    def substitution_cost(c1, c2):
        if c1 == c2:
            return 0
        return levenshtein(decompose(c1), decompose(c2))/3

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            #문자 삽입, 제거는 동일함
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1 
            # Changed (문자 변경 비용을 substitution_cost로 정의한 값으로 사용)
            substitutions = previous_row[j] + substitution_cost(c1, c2)
            current_row.append(min(insertions, deletions, substitutions))

        if debug:
            print(['%.3f'%v for v in current_row[1:]])

        previous_row = current_row

    return previous_row[-1]

In [17]:
s1 = '아이쿠야'
s2 = '아이쿵야'
jamo_levenshtein(s1, s2, debug=True)

['0.000', '1.000', '2.000', '3.000']
['1.000', '0.000', '1.000', '2.000']
['2.000', '1.000', '0.333', '1.333']
['3.000', '2.000', '1.333', '0.333']


0.3333333333333333

d[2,2] = 0.333 이다. ‘쿠’에서 ‘쿵’으로 변하는 비용이 0.333 이다.

### 처음부터 초/중/종성을 모두 분해하여 그대로 Levenshtein distance를 적용해도 된다.

In [18]:
s1 = '아이쿠야'
s2 = '아이쿵야'

s1_ = ''.join([comp for c in s1 for comp in decompose(c)])
s2_ = ''.join([comp for c in s2 for comp in decompose(c)])

print(s1_) # ㅇㅏ ㅇㅣ ㅋㅜ ㅇㅑ 
print(s2_) # ㅇㅏ ㅇㅣ ㅋㅜㅇㅇㅑ 
print(levenshtein(s1_, s2_)/3) # 0.3333333333333333

ㅇㅏ ㅇㅣ ㅋㅜ ㅇㅑ 
ㅇㅏ ㅇㅣ ㅋㅜㅇㅇㅑ 
0.3333333333333333


In [19]:
s1 = '아이쿵야'
s2 = '훍앜이쿠야'

s1_ = ''.join([comp for c in s1 for comp in decompose(c)])
s2_ = ''.join([comp for c in s2 for comp in decompose(c)])

print(s1_) # ㅇㅏ ㅇㅣ ㅋㅜㅇㅇㅑ 
print(s2_) # ㅎㅜㄺㅇㅏㅋㅇㅣ ㅋㅜ ㅇㅑ 
print(levenshtein(s1_, s2_)/3) # 1.6666666666666667

ㅇㅏ ㅇㅣ ㅋㅜㅇㅇㅑ 
ㅎㅜㄺㅇㅏㅋㅇㅣ ㅋㅜ ㅇㅑ 
1.6666666666666667


## soynlp 라이브러리

In [22]:
from soynlp.hangle import levenshtein
from soynlp.hangle import jamo_levenshtein

s1 = '아이쿠야'
s2 = '아이쿵야'

print(levenshtein(s1, s2)) # 1
print(jamo_levenshtein(s1, s2)) # 0.3333333333333333

1
0.3333333333333333
